"By understanding when and why shark attacks occur, it is possible to lessen the likelihood of these incidents."
This database aims to categorize shark attacks, bringing useful information to visualize patterns of causes and effects.

In [3]:
import pandas as pd

In [4]:
# The file wouldn't read properly, returning this error :
# "UnicodeDecodeError: 'utf8' codec can't decode byte 0x9c"
# a quick google search gave me an answer: adding "engine = "python"" to the read code.

In [5]:
df= pd.read_csv('C:\\Users\\Arty\\Desktop\\julia\\IronHack\\SharkAttack\\GSAF5.csv\\GSAF5.csv', engine='python')


In [6]:
#Let's examine the data quickly.
df.shape #24 columns and almost 6000 rows.

(5992, 24)

In [7]:
# The "describe" method doesn't return a lot of information, since the df houses mostly str data.
df.describe()

,Year,original order
count,5992.000000,5992.000000
mean,1925.204606,2997.548899
std,286.473712,1729.860210
min,0.000000,2.000000
25%,1942.000000,1499.750000
50%,1975.000000,2997.500000
75%,2003.000000,4495.250000
max,2016.000000,5993.000000


In [8]:
df.dtypes

Case Number               object
Date                      object
Year                       int64
Type                      object
Country                   object
Area                      object
Location                  object
Activity                  object
Name                      object
Sex                       object
Age                       object
Injury                    object
Fatal (Y/N)               object
Time                      object
Species                   object
Investigator or Source    object
pdf                       object
href formula              object
href                      object
Case Number.1             object
Case Number.2             object
original order             int64
Unnamed: 22               object
Unnamed: 23               object
dtype: object

In [9]:
# Trying to understand what kind of data is in the df.
df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

'Case Number' :case number is composed of the date, followed by a letter, if there were multiple cases on the same day. ex : 2016.09.18.c

'Date' : date, with the format dd-mmm-yy    ex: 17-Sep-16

'Year' : yyyy, int format 

'Type' :  5 types, see below.

'Country','Area' ,'Location' : self explanatory

'Activity'

'Name' : not always a name, sometimes just "male" or "a native" or "unknown"

'Sex ' : 
'Age'
'Injury'
'Fatal (Y/N)'
'Time'
'Species '
'Investigator or Source'

'pdf': name of the pdf file containing the attack data

'href formula' : url to the pdf

'href' : also url

'Case Number.1' : seems to be the same as "Case Number", we will check later on.

'Case Number.2' : the same as above

'original order' : order of cases added to the db

'Unnamed: 22', : seems to be completely empty, probably added by mistake. We'll check later on.

'Unnamed: 23' : same as above.

In [10]:
# I checked the columns with "value_counts" to have an overview of the different data they contain.
df.Type.value_counts()

Unprovoked      4386
Provoked         557
Invalid          519
Sea Disaster     220
Boat             200
Boating          110
Name: Type, dtype: int64

In [11]:
df.Sex .value_counts()
#Trying to check the "Sex" column returns an error, as the name has a space at the end.
# I'll clean the column names to access them easier later on.

AttributeError: 'DataFrame' object has no attribute 'Sex'

# Cleaning the column names

The column names are inconsistent : blank spaces, punctuation, lower and upper case letters...
These are difficult to work with, so I will clean  them.

In [12]:
df.columns = df.columns.str.strip().str.replace(' ', '_').str.replace(":","").str.replace("(","").str.replace(")","")
df.columns = df.columns.str.replace("/","").str.lower().str.replace(".","")
df.columns

Index(['case_number', 'date', 'year', 'type', 'country', 'area', 'location',
       'activity', 'name', 'sex', 'age', 'injury', 'fatal_yn', 'time',
       'species', 'investigator_or_source', 'pdf', 'href_formula', 'href',
       'case_number1', 'case_number2', 'original_order', 'unnamed_22',
       'unnamed_23'],
      dtype='object')

In [13]:
df.Sex.value_counts() #now it works !

AttributeError: 'DataFrame' object has no attribute 'Sex'

From this quick overview, the next steps could be:

- check if the columns case_number, case_number1, case_number2 contain identical data, if they do, drop 2 of them.
- check if 'href_formula' and 'href' contain identical data. If they do, drop one of them.
- check if unnamed_22 and unnamed_23 contain any data. If they don't, drop them.
- it could be interesting to convert the data from the "date", and "time" columns to datetime types.
- Dropping unnecessary columns.

# Unnecessary columns (information that isn't useful)

The "name" column doesn't add any info to the nature of the attacks. 
I shall drop it.
ALso, the "original order" column doesn't seem necessary, since some rows are going to be deleted.
"investigator_or_source" lists the press coverage of the event. This source is also cited in the pdf of the event, accessible by the link in "href". I will drop "invetigator_or_source"

In [14]:
df = df.drop("name", axis=1)

In [15]:
df = df.drop("original_order", axis=1)

In [16]:
df = df.drop("investigator_or_source", axis=1)

In [17]:
df.head()

,case_number,date,year,type,country,area,location,activity,sex,age,...,fatal_yn,time,species,pdf,href_formula,href,case_number1,case_number2,unnamed_22,unnamed_23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,16,...,N,13h00,NaN,2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,36,...,N,11h00,NaN,2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,43,...,N,10h43,NaN,2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,M,NaN,...,N,NaN,NaN,2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,M,NaN,...,N,NaN,2 m shark,2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,NaN,NaN


# Checking if the suspected columns contain identical data, and dropping the double up columns.

In [18]:
same = (df['case_number']==df['case_number2'])
same.value_counts()
#We quickly see that "case_number" and "case_number2" contain almost identical data.
#Let's verify this assumption.
perc = round((same.value_counts() / len(same))*100,2)
perc
#These columns contain identical value, for 99,97%. Let's drop column case_number2.

True     99.97
False     0.03
dtype: float64

In [19]:
df = df.drop("case_number2", axis=1)

In [20]:
#Let's do the same with the other columns that might contain identical data.
same = (df['case_number']==df['case_number1'])
same.value_counts()
perc = round((same.value_counts() / len(same))*100,2)
#perc is over 99% again, so we're dropping "case_number1"

In [21]:
df = df.drop("case_number1", axis=1)

In [22]:
same = (df['href_formula']==df['href'])
same.value_counts()
perc = round((same.value_counts() / len(same))*100,2)
#Again, over 99%, so we're dropping one of the 2.
df = df.drop("href_formula", axis=1)

# Checking if the "unnamed_22" and 23 columns contain any data, if not, dropping them.

In [23]:
#Lets get a quick overview of all the NaN percentages.
null_cols = df.isnull().sum()
#only for the columns that contain NaN
null_perc=round(null_cols[null_cols>0]/df.shape[0]*100,2)
null_perc
#We see that the "unnamed" columns are over 99% empty. Let's drop them.

country        0.72
area           6.71
location       8.28
activity       8.80
sex            9.46
age           44.74
injury         0.45
fatal_yn       0.32
time          53.62
species       48.97
href           0.05
unnamed_22    99.98
unnamed_23    99.97
dtype: float64

In [24]:
df = df.drop("unnamed_22", axis=1)
df = df.drop("unnamed_23", axis=1)

In [25]:
# Also going to drop "pdf" as the href column provides a direct link to the pdf.

In [26]:
df =df.drop("pdf", axis=1)

# Duplicate rows ?

We've checked the columns for duplicates and NaN. Now let's check if there are any duplicate rows.

In [27]:
before = len(df)
df =df.drop_duplicates()
after = len(df)
print("Number of duplicates dropped:", str(before-after))

#The df didn't contain any exact duplicate rows. However, this is very rare in any case.

Number of duplicates dropped: 0


# Numerical data

Let's check what kind of numerical data we have.

In [28]:
df.dtypes

case_number    object
date           object
year            int64
type           object
country        object
area           object
location       object
activity       object
sex            object
age            object
injury         object
fatal_yn       object
time           object
species        object
href           object
dtype: object

# Text errors

In [29]:
df.fatal_yn.value_counts()
#The data in "fatal" isn't homogeneous. Let's clean it.

N          4315
Y          1552
UNKNOWN      94
 N            8
#VALUE!       1
F             1
n             1
N             1
Name: fatal_yn, dtype: int64

In [30]:
df.fatal_yn=df.fatal_yn.str.strip().str.upper()
df.fatal_yn.value_counts()

N          4325
Y          1552
UNKNOWN      94
#VALUE!       1
F             1
Name: fatal_yn, dtype: int64

In [31]:
# I want to drop the rows where "fatal_yn" isn't Y or N.
df = df[df["fatal_yn"].isin(["Y","N"])]


In [32]:
df.shape

(5877, 15)

In [33]:
# Cleaning the country column values

In [34]:
df.country= df.country.str.strip(" ?").str.upper()
df.country.value_counts()

USA                               2088
AUSTRALIA                         1251
SOUTH AFRICA                       559
PAPUA NEW GUINEA                   131
NEW ZEALAND                        122
BRAZIL                             100
BAHAMAS                             98
MEXICO                              82
ITALY                               66
FIJI                                64
PHILIPPINES                         58
REUNION                             57
NEW CALEDONIA                       51
MOZAMBIQUE                          43
CUBA                                41
SPAIN                               38
EGYPT                               36
PANAMA                              32
JAPAN                               32
INDIA                               31
CROATIA                             30
IRAN                                29
SOLOMON ISLANDS                     28
HONG KONG                           24
GREECE                              24
JAMAICA                  

In [35]:
import numpy as np

# Looping to strip all columns at once

because I don't really have to do the same thing 20 times

In [36]:
for col in df.select_dtypes([np.object]):
    df[col] = df[col].str.strip(" \".")

In [37]:
df.head()

,case_number,date,year,type,country,area,location,activity,sex,age,injury,fatal_yn,time,species,href
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,16,Minor injury to thigh,N,13h00,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,36,Lacerations to hands,N,11h00,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,43,Lacerations to lower leg,N,10h43,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,M,NaN,Struck by fin on chest & leg,N,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,M,NaN,No injury: Knocked off board by shark,N,NaN,2 m shark,http://sharkattackfile.net/spreadsheets/pdf_di...


# Checking NaNs :

In [38]:
null_cols = df.isnull().sum()
nan_perc=round(null_cols[null_cols>0]/df.shape[0]*100,2)
nan_perc

country      0.68
area         6.55
location     8.10
activity     8.01
sex          8.83
age         43.97
time        52.94
species     48.48
href         0.05
dtype: float64

In [39]:
#I will drop the rows with NaN values in the columns with <1% NaN.
null_drop = nan_perc[nan_perc<1].index  #this is the list of columns where I'll drop the NaN rows
df= df.dropna(subset=['country', 'investigator_or_source', 'href'])
df.shape

KeyError: ['investigator_or_source']

In [40]:
null_cols = df.isnull().sum()
nan_perc=round(null_cols[null_cols>0]/df.shape[0]*100,2)
nan_perc

country      0.68
area         6.55
location     8.10
activity     8.01
sex          8.83
age         43.97
time        52.94
species     48.48
href         0.05
dtype: float64

In [41]:
# I tried dropping rows using the "thresh" argument in the dropna method, 
# testing if there were rows with more than one "NaN".
# There were no such rows.

# Replacing NaNs

In [42]:
df_test = df["time"].fillna("NOT AVAILABLE")
df_test.value_counts()

NOT AVAILABLE                           3111
Afternoon                                174
11h00                                    122
Morning                                  108
12h00                                    107
15h00                                    100
14h00                                     94
16h00                                     92
16h30                                     72
13h00                                     70
14h30                                     70
17h30                                     69
17h00                                     63
15h30                                     62
18h00                                     61
11h30                                     61
Night                                     58
13h30                                     56
10h00                                     53
09h00                                     46
10h30                                     45
Evening                                   34
12h30     

In [43]:
#Testing how to replace NaNs, I almost replaced my whole dataset with a series 
#(keeping only the column i was working on)

In [44]:
nan_cols = df.columns[df.isna().any()].tolist()
df[nan_cols] = df[nan_cols].fillna("NOT AVAILABLE")

In [45]:
df.head()

,case_number,date,year,type,country,area,location,activity,sex,age,injury,fatal_yn,time,species,href
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,16,Minor injury to thigh,N,13h00,NOT AVAILABLE,http://sharkattackfile.net/spreadsheets/pdf_di...
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,36,Lacerations to hands,N,11h00,NOT AVAILABLE,http://sharkattackfile.net/spreadsheets/pdf_di...
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,43,Lacerations to lower leg,N,10h43,NOT AVAILABLE,http://sharkattackfile.net/spreadsheets/pdf_di...
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,M,NOT AVAILABLE,Struck by fin on chest & leg,N,NOT AVAILABLE,NOT AVAILABLE,http://sharkattackfile.net/spreadsheets/pdf_di...
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,M,NOT AVAILABLE,No injury: Knocked off board by shark,N,NOT AVAILABLE,2 m shark,http://sharkattackfile.net/spreadsheets/pdf_di...


# Cleaning the "time" column, grouping times by moments of the day.

In [46]:
df["time"] = df["time"].str.replace("h",":")
df.head()

,case_number,date,year,type,country,area,location,activity,sex,age,injury,fatal_yn,time,species,href
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,16,Minor injury to thigh,N,13:00,NOT AVAILABLE,http://sharkattackfile.net/spreadsheets/pdf_di...
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,36,Lacerations to hands,N,11:00,NOT AVAILABLE,http://sharkattackfile.net/spreadsheets/pdf_di...
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,43,Lacerations to lower leg,N,10:43,NOT AVAILABLE,http://sharkattackfile.net/spreadsheets/pdf_di...
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,M,NOT AVAILABLE,Struck by fin on chest & leg,N,NOT AVAILABLE,NOT AVAILABLE,http://sharkattackfile.net/spreadsheets/pdf_di...
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,M,NOT AVAILABLE,No injury: Knocked off board by shark,N,NOT AVAILABLE,2 m shark,http://sharkattackfile.net/spreadsheets/pdf_di...


In [47]:
df.time.value_counts()

NOT AVAILABLE              3111
Afternoon                   174
11:00                       122
Morning                     108
12:00                       107
15:00                       100
14:00                        94
16:00                        92
16:30                        72
13:00                        70
14:30                        70
17:30                        69
17:00                        63
15:30                        62
18:00                        61
11:30                        61
Nig:t                        58
13:30                        56
10:00                        53
09:00                        46
10:30                        45
Evening                      34
12:30                        33
Late afternoon               31
09:30                        31
07:30                        30
08:00                        28
18:30                        27
08:30                        25
19:00                        23
                           ... 
09:11   

In [48]:
# Trying to use a function I wrote to replace times with moments of the day.

In [49]:
import re

def timereplace(f):
    if re.search("[0][5-7]:",f):
        f = "early morning"
    elif (re.search("[0][8-9]:",f)) or (re.search("[1][0]:",f)):
        f = "morning"
    elif re.search("[1][1-2]:",f):
        f = "late morning"
    elif re.search("[1][3-7]:",f):
        f= "afternoon"
    elif re.search("[1][8-9]:",f):
        f = "evening"
    elif re.search("[2][0-2]:",f):
        f = "early night"
    elif re.search("[2][3]:",f) or (re.search("[0][0-4]:",f)):
        f = "night"
    return f

In [50]:
df["time"] = df["time"].apply(timereplace)
df["time"] = df["time"].str.strip("")
df["time"] = df["time"].str.replace("Late afternoon", "late afternoon")
df["time"] = df["time"].str.replace("Afternoon", "afternoon")
df["time"] = df["time"].str.replace("Morning", "morning")
df["time"] = df["time"].str.replace("Nig:t", "night")


In [51]:
df["time"] = df["time"].str.replace("Dusk", "evening")
df["time"] = df["time"].str.replace("Mid afternoon", "afternoon")
df["time"] = df["time"].str.replace("Dawn", "early morning")

In [52]:
#I applied this sort of replacement just by observing the remaining values.

In [53]:
timevalues = ["NOT AVAILABLE", "afternoon", "morning", "early morning", "late morning", "evening", "night", "early night", "late afternoon","early afternoon"]

In [54]:
df.time.value_counts()

NOT AVAILABLE                                                            3111
afternoon                                                                1229
morning                                                                   472
late morning                                                              446
evening                                                                   183
early morning                                                             125
night                                                                      92
early night                                                                38
Evening                                                                    34
late afternoon                                                             31
P.M                                                                        12
Early morning                                                              12
A.M                                                             

In [55]:
indexNames = df[(~df["time"].isin(timevalues))].index  #returning index of rows with unusable time
df.drop(indexNames , inplace=True)

In [56]:
df.time.value_counts()

NOT AVAILABLE     3111
afternoon         1229
morning            472
late morning       446
evening            183
early morning      125
night               92
early night         38
late afternoon      31
Name: time, dtype: int64

Time column is clean.

In [57]:
df.head()

#Columns to deal with : area, location, age, injury, species

,case_number,date,year,type,country,area,location,activity,sex,age,injury,fatal_yn,time,species,href
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,16,Minor injury to thigh,N,afternoon,NOT AVAILABLE,http://sharkattackfile.net/spreadsheets/pdf_di...
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,36,Lacerations to hands,N,late morning,NOT AVAILABLE,http://sharkattackfile.net/spreadsheets/pdf_di...
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,43,Lacerations to lower leg,N,morning,NOT AVAILABLE,http://sharkattackfile.net/spreadsheets/pdf_di...
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,M,NOT AVAILABLE,Struck by fin on chest & leg,N,NOT AVAILABLE,NOT AVAILABLE,http://sharkattackfile.net/spreadsheets/pdf_di...
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,M,NOT AVAILABLE,No injury: Knocked off board by shark,N,NOT AVAILABLE,2 m shark,http://sharkattackfile.net/spreadsheets/pdf_di...


# Dealing with the activity column

In [58]:
df["activity"]=df["activity"].str.strip().str.lower().str.replace("-"," ")
df.activity.value_counts()


surfing                                                                                                   867
swimming                                                                                                  827
not available                                                                                             468
fishing                                                                                                   411
spearfishing                                                                                              316
bathing                                                                                                   144
wading                                                                                                    138
diving                                                                                                    117
standing                                                                                                   97
scuba divi

In [59]:
def replaceactivity(mystring):
    if ("body surfing" in mystring) or ("boogie boarding" in mystring):
        mystring = "body surfing"
    elif "surf" in mystring:
        mystring = "surfing"
    elif ("walking" in mystring) or ("paddling" in mystring):
        mystring = "wading"
    elif "treading water" in mystring:
        mystring="swimming"
    elif ("fall" in mystring) or ("fell" in mystring) or ("rescue" in mystring) or ("capsize" in mystring) or ("shipwreck" in mystring) or ("adrift"in mystring):
        mystring = "sea disaster"
    elif ("kayak" in mystring) or ("canoe" in mystring) or ("rowing" in mystring):
        mystring = "in row boat"
    elif ("diving for" in mystring) or ("pearl" in mystring):
        mystring = "diving for fish/pearls"
    elif ("spearfish" in mystring):
        mystring = "spearfishing"
    elif ("snorkel" in mystring):
        mystring = "snorkelling"
    elif ("fishing for" in mystring):
        mystring = "fishing"
    elif ("floating" in mystring):
        mystring = "floating"
    return mystring

In [60]:
df["activity"] = df["activity"].apply(replaceactivity)

In [61]:
# Making activities more homogenous.

In [62]:
df.activity.value_counts()

surfing                                                                                                         1083
swimming                                                                                                         866
fishing                                                                                                          476
not available                                                                                                    468
spearfishing                                                                                                     395
sea disaster                                                                                                     207
wading                                                                                                           166
bathing                                                                                                          144
diving for fish/pearls                                          

# Dealing with age column

In [63]:
df.age.value_counts()

NOT AVAILABLE              2536
17                          142
18                          140
16                          132
20                          132
19                          132
15                          130
22                          110
21                          109
24                           98
25                           95
14                           94
13                           88
27                           76
26                           76
23                           75
29                           73
28                           73
30                           71
12                           68
35                           59
32                           59
40                           50
10                           49
31                           47
38                           46
34                           45
43                           42
36                           41
33                           40
                           ... 
86      

In [64]:
def agereplace(agestr):
    if re.search("[0-9][0-9]",agestr):    #checking if there is a match with double digit age
        m = re.search("[0-9][0-9]", agestr)  # if there is, return the match
        result = m.group(0) 
        agestr = result #switch the original text to the match
    return agestr

In [65]:
df.head()

,case_number,date,year,type,country,area,location,activity,sex,age,injury,fatal_yn,time,species,href
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",surfing,M,16,Minor injury to thigh,N,afternoon,NOT AVAILABLE,http://sharkattackfile.net/spreadsheets/pdf_di...
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",surfing,M,36,Lacerations to hands,N,late morning,NOT AVAILABLE,http://sharkattackfile.net/spreadsheets/pdf_di...
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",surfing,M,43,Lacerations to lower leg,N,morning,NOT AVAILABLE,http://sharkattackfile.net/spreadsheets/pdf_di...
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,surfing,M,NOT AVAILABLE,Struck by fin on chest & leg,N,NOT AVAILABLE,NOT AVAILABLE,http://sharkattackfile.net/spreadsheets/pdf_di...
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,surfing,M,NOT AVAILABLE,No injury: Knocked off board by shark,N,NOT AVAILABLE,2 m shark,http://sharkattackfile.net/spreadsheets/pdf_di...


In [66]:
df["age"] = df["age"].apply(agereplace)

In [67]:
df["age"].value_counts()

NOT AVAILABLE      2536
17                  144
18                  143
20                  139
19                  133
16                  133
15                  130
21                  112
22                  110
24                   98
25                   97
14                   95
13                   90
30                   80
23                   77
26                   76
27                   76
28                   75
29                   73
12                   70
32                   60
35                   59
10                   53
40                   52
31                   49
38                   46
34                   46
33                   44
36                   43
43                   42
                   ... 
64                    3
66                    3
77                    3
78                    2
75                    2
74                    2
F                     2
2 to 3 months         1
middle-age            1
81                    1
7 or 8          

In [68]:
# 1st big mistake: completely emptying the age column. Filled with None. Not sure how ??

In [72]:
df["age"] = df["age"].apply(agereplace2)
df["age"].value_counts()

NOT AVAILABLE      2536
18-24               812
25-34               676
12-17               675
35-44               413
45-54               245
under 12            197
55-64               110
65-74                35
75 and over          11
young                 3
                      3
F                     2
                      1
X                     1
middle-age            1
(adult)               1
A.M                   1
M                     1
MAKE LINE GREEN       1
adult                 1
Elderly               1
Name: age, dtype: int64

In [73]:
df["age"]= df["age"].str.replace("Teens","12-17").str.replace("Teen","12-17").str.replace("teen","12-17")

In [74]:
df["age"]= df["age"].str.replace("young","12-17").str.replace("Teen","12-17").str.replace("teen","12-17")

In [75]:
df["age"]= df["age"].str.strip(" \"()")

In [76]:
agegroups = ["NOT AVAILABLE","18-24", "12-17","25-34","35-44","45-54","under 12","55-64","65-74","75 and over"]

In [77]:
indexNames = df[(~df["age"].isin(agegroups))].index  #returning index of rows with unusable time
df.drop(indexNames , inplace=True)

In [78]:
df.age.value_counts()

NOT AVAILABLE    2536
18-24             812
12-17             678
25-34             676
35-44             413
45-54             245
under 12          197
55-64             110
65-74              35
75 and over        11
Name: age, dtype: int64

In [79]:
# Age column is clean.

In [80]:
def agereplace2(agestr):
    if re.search("[1-1][2-7]",agestr):     
        agestr = "12-17" #switch the original text to the match
    elif re.search("[1-1][8-9]",agestr) or re.search("[2-2][0-4]",agestr) : 
        agestr = "18-24" 
    elif re.search("[2-2][5-9]",agestr) or re.search("[3-3][0-4]",agestr):    
        agestr = "25-34"
    elif re.search("[3-3][5-9]",agestr) or re.search("[4-4][0-4]",agestr):    
        agestr = "35-44"
    elif re.search("[4-4][5-9]",agestr) or re.search("[5-5][0-4]",agestr) :     
        agestr = "45-54"
    elif re.search("[5-5][5-9]",agestr) or re.search("[6-6][0-4]",agestr):    
        agestr = "55-64"
    elif re.search("[6-6][5-9]",agestr) or re.search("[7-7][0-4]",agestr):    
        agestr = "65-74"
    elif re.search("[7-7][5-9]",agestr) or re.search("[8-9][0-9]",agestr) :    
        agestr = "75 and over"
    elif re.search("[\d\b]",agestr) or re.search("[1-1][0-1]",agestr):    
        agestr = "under 12"
    return agestr

Ideas:


Grouping geographical locations : trying to use geograpy library, had to import nltk, debugging installation etc, but doesn't seem to work.



In [81]:
df.head()

,case_number,date,year,type,country,area,location,activity,sex,age,injury,fatal_yn,time,species,href
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",surfing,M,12-17,Minor injury to thigh,N,afternoon,NOT AVAILABLE,http://sharkattackfile.net/spreadsheets/pdf_di...
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",surfing,M,35-44,Lacerations to hands,N,late morning,NOT AVAILABLE,http://sharkattackfile.net/spreadsheets/pdf_di...
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",surfing,M,35-44,Lacerations to lower leg,N,morning,NOT AVAILABLE,http://sharkattackfile.net/spreadsheets/pdf_di...
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,surfing,M,NOT AVAILABLE,Struck by fin on chest & leg,N,NOT AVAILABLE,NOT AVAILABLE,http://sharkattackfile.net/spreadsheets/pdf_di...
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,surfing,M,NOT AVAILABLE,No injury: Knocked off board by shark,N,NOT AVAILABLE,2 m shark,http://sharkattackfile.net/spreadsheets/pdf_di...


# Injury column

Dropping column but replacing it with 2 news columns : body part attacked AND type of injury.
"Fatality" is already covered in "FATAL_YN" column.

In [82]:
df.injury.value_counts()

FATAL                                                                                            714
Survived                                                                                          96
Foot bitten                                                                                       80
No injury                                                                                         74
Leg bitten                                                                                        72
Left foot bitten                                                                                  49
Right foot bitten                                                                                 39
Hand bitten                                                                                       30
No injury, board bitten                                                                           27
Thigh bitten                                                                               

In [83]:
def bodypart(strg):
    lower_limb = ["shin","foot", "feet", "ankle", "knee", "leg", "thigh", "calf", "calves","heel","toe", "buttock"]
    upper_limb = ["hand", "arm", "finger", "wrist", "elbow", "shoulder"]
    abdomen = ["torso", "abdomen","chest","groin","hip"]
    head = ["head", "neck", "face","cheek"]
    lst = [lower_limb, upper_limb, abdomen, head]
    name_list = ["lower limbs", "upper limbs", "abdomen", "head or neck"]
    import re
    
    for i in range(len(lst)):
        if re.compile('|'.join(lst[i]),re.IGNORECASE).search(strg): 
            strg = name_list[i]
    if "no injury" in strg :
        strg = "no injury"
    
    return strg

In [84]:
df["body_area_attacked"] = df["injury"].str.lower().apply(bodypart)

In [85]:
df.body_area_attacked.value_counts()

lower limbs                                                                                                                                                                        2426
upper limbs                                                                                                                                                                         838
fatal                                                                                                                                                                               715
no injury                                                                                                                                                                           696
abdomen                                                                                                                                                                             112
survived                                                                        

In [86]:
def bodypart2(strg):
    
    name_list = ["lower limbs", "upper limbs", "abdomen", "head or neck", "no injury"]
    import re
    
    if not re.compile('|'.join(name_list),re.IGNORECASE).search(strg): 
        strg = "NOT AVAILABLE"

    
    return strg

In [87]:
df["body_area_attacked"] = df["body_area_attacked"].apply(bodypart2)

In [88]:
df.head()

,case_number,date,year,type,country,area,location,activity,sex,age,injury,fatal_yn,time,species,href,body_area_attacked
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",surfing,M,12-17,Minor injury to thigh,N,afternoon,NOT AVAILABLE,http://sharkattackfile.net/spreadsheets/pdf_di...,lower limbs
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",surfing,M,35-44,Lacerations to hands,N,late morning,NOT AVAILABLE,http://sharkattackfile.net/spreadsheets/pdf_di...,upper limbs
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",surfing,M,35-44,Lacerations to lower leg,N,morning,NOT AVAILABLE,http://sharkattackfile.net/spreadsheets/pdf_di...,lower limbs
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,surfing,M,NOT AVAILABLE,Struck by fin on chest & leg,N,NOT AVAILABLE,NOT AVAILABLE,http://sharkattackfile.net/spreadsheets/pdf_di...,lower limbs
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,surfing,M,NOT AVAILABLE,No injury: Knocked off board by shark,N,NOT AVAILABLE,2 m shark,http://sharkattackfile.net/spreadsheets/pdf_di...,no injury


In [89]:
# Creating a "type of injury" column.

In [90]:
def typeinjury(strg):
    
    if "bit" in strg :
        strg = "bite"
    elif "lacerat" in strg :
        strg = "lacerations"
    elif "severed" in strg:
        strg = "severed body part"
    elif "no injury" in strg:
        strg = "no injury"
    elif "strike" in strg or "struck" in strg:
        strg = "strike"
    else :
        strg = "NOT AVAILABLE"
    
    return strg

In [91]:
df["type_of_injury"] = df["injury"].str.lower().apply(typeinjury)

In [92]:
df.type_of_injury.value_counts()

NOT AVAILABLE        2034
bite                 1753
lacerations          1195
no injury             506
severed body part     214
strike                 11
Name: type_of_injury, dtype: int64

In [93]:
df = df.drop("injury", axis=1)

In [94]:
df.date.value_counts()

1957                         11
1942                          9
1956                          7
1958                          7
1950                          7
1949                          6
1941                          6
No date                       6
1959                          5
12-Apr-01                     5
No date, Before 1963          5
Oct-60                        5
28-Jul-95                     5
Aug-56                        5
1954                          5
1970s                         5
05-Oct-03                     5
1938                          4
09-Jul-94                     4
14-Jun-12                     4
1961                          4
27-Jul-52                     4
20-Sep-15                     4
1898                          4
1960s                         4
27-Dec-08                     4
1952                          4
1960                          4
1876                          4
23-Jan-70                     4
                             ..
04-Oct-9

# Reorganizing and renaming columns

In [287]:
df.columns = df.columns.str.upper()

In [290]:
df = df.rename(columns={'TYPE': 'PROVOCATION'})

In [291]:
df.columns

Index(['CASE_NUMBER', 'DATE', 'YEAR', 'PROVOCATION', 'COUNTRY', 'AREA',
       'LOCATION', 'ACTIVITY', 'SEX', 'AGE', 'FATAL_YN', 'TIME', 'SPECIES',
       'HREF', 'BODY_AREA_ATTACKED', 'TYPE_OF_INJURY'],
      dtype='object')

In [293]:
df.shape

(5712, 16)

In [295]:
df = df[["CASE_NUMBER","HREF","YEAR", "DATE","TIME","COUNTRY","AREA","LOCATION","SPECIES","PROVOCATION","ACTIVITY",'SEX', 'AGE', 'FATAL_YN','BODY_AREA_ATTACKED', 'TYPE_OF_INJURY' ]]

# Exporting to csv

In [296]:
df.to_csv(r"C:\Users\Arty\Desktop\julia\IronHack\Docs pedago\data-labs\module-1\pandas-project\your-code\pandas_sharks.csv")